In [ ]:
#| default_exp common.user

In [ ]:
#| export
import time
from functools import wraps
from dataclasses import dataclass
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
import fastlite as fl
import fastcore.all as fc
from meme_games.common.utils import *

In [ ]:

# | export
@dataclass
class User:
    uid: str
    name: str
    filename: Optional[str] = None

    async def set_picture(self, file: UploadFile, content_path='./user-content'):
        self.reset_picture(content_path)
        fname = f"{self.uid}_{time.time()}.{file.filename.split('.')[-1]}"
        dir = Path(content_path)
        dir.mkdir(parents=True, exist_ok=True)
        with open(dir/fname, 'wb') as f: f.write(await file.read())
        self.filename = str(fname)

    def reset_picture(self, content_path='./user-content'):
        if not self.filename: return
        path = Path(content_path)/self.filename
        path.unlink(missing_ok=True)
        self.filename = None

In [ ]:
#| export
@fc.delegates(Database)
def init_db(filename_or_conn=':memory:', **kwargs):
    db = Database(filename_or_conn, **kwargs)
    db.create(User, pk='uid', transform=True)
    db.t.user.cls = User
    return db

In [ ]:
#| export
class UserManager(DataManager):
    def __post_init__(self): self.users: fl.Table = self.db.t.user
    
    def create(self, uid: str=None, name: str = 'null'):
        u = User(uid or random_id(), name)
        self.users.insert(u)
        return u
    
    def get(self, uid: str): return self.users.get(uid)
    def update(self, user: User): return self.users.update(user)
    
    def get_or_create(self, sess: dict, name: str = 'null') -> User:
        sess = sess['session'] if 'session' in sess else sess
        uid = sess.setdefault('uid', random_id())
        if uid in self.users: return self.users[uid]
        return self.create(uid, name)

In [ ]:
#| export
def user_beforeware(manager: UserManager, skip=None):
    '''Makes sure that request always contains valid user'''
    def before(req: Request): req.state.user = manager.get_or_create(req.session)
    return Beforeware(before, skip)

In [ ]:
# | export

def UserName(u: User, is_connected=True, cls='username', **kwargs):
    cls += ' muted' if not is_connected else ''
    return Span(u.name, dt_user = u.uid, cls=cls, **kwargs, hx_swap_oob=f"outerHTML:span[dt-user='{u.uid}']")

def NameSetting():
    return Div(
        I('edit', cls="material-icons"),
        Form(Input(type='hidden', name='name'), hx_put='/name', hx_swap='none'),
        _=r"""on click get prompt('Enter your name')
                 if it != null then if it.trim() != "" then set x to it
                 make RegExp from '\\s+', 'g' called myregx
                 set x to x.replace(myregx, ' ').trim() then log x
                 put x into @value of <form input/> in me then trigger submit on <form/> in me
                 """, cls='controls')

In [ ]:
db = init_db()
user_manager = UserManager(db)
bware = user_beforeware(user_manager)

In [ ]:
hdrs = [
    Script(src="https://unpkg.com/hyperscript.org@0.9.13"),
    Link(rel="stylesheet", href="https://fonts.googleapis.com/icon?family=Material+Icons")
]
app, rt = fast_app(pico=False, before=bware, hdrs=hdrs)

In [ ]:
@rt
def index(req: Request):
    return Titled(f"Hello {req.state.user.name}!", NameSetting())

In [ ]:
server = JupyUvi(app)

In [ ]:
server.stop()